# HTTP request to ZhipuAI GLM API

**This tutorial is available in English and is attached below the Chinese explanation**

首先，从最简单的 http 请求开始访问 ZhipuAI 的 GLM 模型的API。 填写完API_KEY后，按照[官方文档](https://open.bigmodel.cn/dev/api#nosdk)的步骤，设计一个简单的http鉴权请求。

First, start with the simplest http request to access the API of the ZhipuAI's GLM model. After filling in the API_KEY, follow the steps of [Official Document](https://open.bigmodel.cn/dev/api#nosdk) to design a simple http authentication request.

In [1]:
import os
os.environ["ZHIPUAI_API_KEY"] = "your api key"

In [2]:
import requests
import jwt
import time

def generate_token(apikey: str, exp_seconds: int):
    try:
        id, secret = apikey.split(".")
    except Exception as e:
        raise Exception("invalid apikey", e)

    payload = {
        "api_key": id,
        "exp": int(round(time.time() * 1000)) + exp_seconds * 1000,
        "timestamp": int(round(time.time() * 1000)),
    }

    return jwt.encode(
        payload,
        secret,
        algorithm="HS256",
        headers={"alg": "HS256", "sign_type": "SIGN"},
    )

接着，我们将函数应用到请求中，并直接执行请求

Next, we apply the function to the request and execute the request directly

In [3]:
api_key = os.environ["ZHIPUAI_API_KEY"]
token = generate_token(api_key, 60)
url = "https://open.bigmodel.cn/api/paas/v4/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {token}"
}

data = {
    "model": "glm-4",
    "messages": [
        {
            "role": "system",
            "content": "your are a helpful assistant"
        },
        {
            "role": "user",
            "content": "can you tell me a joke?"
        }
    ],
    "max_tokens": 8192,
    "temperature": 0.8,
    "stream": False
}

response = requests.post(url, headers=headers, json=data)
ans = response.json()

接着，我们就得到模型的返回

Then, we get the return of the model

In [4]:
ans

{'choices': [{'finish_reason': 'stop',
   'index': 0,
   'message': {'content': "Sure! Here's a light-hearted joke for you:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything!",
    'role': 'assistant'}}],
 'created': 1705933810,
 'id': '8313804547540260859',
 'model': 'glm-4',
 'request_id': '8313804547540260859',
 'usage': {'completion_tokens': 26, 'prompt_tokens': 19, 'total_tokens': 45}}

我们可以这样获取返回的内容和ID

We can get the returned content and ID like this

In [5]:
print("content:", ans["choices"][0]["message"]["content"])
print("===============")
print("id:", ans["id"])

content: Sure! Here's a light-hearted joke for you:

Why don't scientists trust atoms?

Because they make up everything!
id: 8313804547540260859


如果我想用流式输出，该怎么办呢？我们只需要将`data`中的 `stream`设置为`True`即可！

What should I do if I want to use streaming output? We just need to set `stream` in `data` to `True`!

In [6]:
import json

api_key = os.environ["API_KEY"]
token = generate_token(api_key, 60)
url = "https://open.bigmodel.cn/api/paas/v4/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {token}"
}

data["stream"] = True
response = requests.post(url, headers=headers, json=data)
for chunk in response.iter_lines():
    if chunk:
        chunk_str = chunk.decode('utf-8')
        json_start_pos = chunk_str.find('{"id"')
        if json_start_pos != -1:
            json_str = chunk_str[json_start_pos:]
            json_data = json.loads(json_str)
            for choice in json_data.get('choices', []):
                delta = choice.get('delta', {})
                content = delta.get('content', '')
                print(content, end='|') # I use | to separate the content

Sure|!| Here|'s| a| light|-hearted| joke| for| you|:

Why| don|'t| scientists| trust| atoms|?

Because| they| make| up| everything|!||